# Importing libraries

In [1]:
from utils.config import read_config

config = read_config('config.yml')

if config.is_experiment:
    
    from comet_ml import Experiment
    experiment = Experiment(
        api_key="VCyOxE7IjP9abOTTSwvn2gfs4",
        project_name="pointcloudfcd",
        workspace="mrugnivenko",
    )

import os
import time
import json
import uuid
import yaml
import utils
import pickle
import shutil
import datetime
import numpy as np
from tqdm import tqdm
from datetime import date
from sklearn.model_selection import KFold

from utils.train_new_2 import *

config = read_config('config.yml')

import warnings
warnings.filterwarnings("ignore")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mrugnivenko/pointcloudfcd/34b2955c797d43adb48dfb978e443837

/usr/local/lib/python3.6/dist-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


# Runing experiment

In [ ]:
# Specifiyng GPU device. 
torch.cuda.set_device(f"cuda:{config.device}")

# Setting name for experimnet.
mode_of_run = 'experiment' if config.is_experiment else 'test'
config.name_of_experiment = '_'.join([str(datetime.date.today()), mode_of_run, str(uuid.uuid4())])
print(''.join([config.name_of_experiment, '\n']))
if config.is_experiment:
    experiment.set_name(config.name_of_experiment)
    
# Specifying subjects used for training.
allowed_subjects = np.load(config.path_to_allowed_subjects, allow_pickle=True).tolist()  
config.subjects = allowed_subjects

# Specifying number of feaures.
config.input_features_dim = 3 + len(config.features)

# Adding dictionaries for noramlization.
MEANS, STDS = {}, {}
for feature in config.features:
    MEANS[feature] = []
    STDS[feature] = []
config.MEANS = MEANS
config.STDS = STDS

# Downloading folds division.
with open(config.path_to_folds) as json_file:
    kf = json.load(json_file)
# for key in kf:
#     kf[key][0] =  kf[key][0][:2]
#     kf[key][1] =  kf[key][1][:1]

# Saving parameters of experiment
if config.is_experiment:
    os.makedirs(f'experiments/{config.name_of_experiment}', exist_ok=True)
    os.makedirs(f'experiments/{config.name_of_experiment}/weights', exist_ok=True)
    config_to_write = dict(config)
    config_to_write['pospool'] = dict(config_to_write['pospool'])
    with open(f'experiments/{config.name_of_experiment}/config.json', 'w') as file:
        json.dump(config_to_write, file)
        
data_dict = {}
for feature in config.features:
    data_dict[feature] = [f"{config.path_to_data}/{feature}/{subject}.nii.gz" for subject in config.subjects]
data_dict['labels'] = [f"{config.path_to_data}/labels/{subject}.nii.gz" for subject in config.subjects]
data_dict['brains'] = [f"{config.path_to_data}/{config.brains_modality}_brains/{subject}.nii.gz" for subject in config.subjects]

torch.cuda.empty_cache()
train(data_dict, kf, config, experiment, config.is_experiment)

  0%|          | 0/5 [00:00<?, ?it/s]

2022-03-29_experiment_39fe01a4-ec9f-460a-a831-3d5ad654c1ec

Weights are: [0.019387950067934784, 0.9806120499320652]
MEANS are: {'brains': 141.8812255859375, 'curv': -0.28408604860305786}
STDS are: {'brains': 113.89356231689453, 'curv': 1.156050682067871}
Train dataset created
Test dataset created
size of train dataset: 46
size of test dataset: 12



 62%|██████▏   | 249/400 [3:18:46<2:00:32, 47.90s/it]